# Detecting Filler Speech

**TODO**:
- Generator + LLM Critic experiment

In [0]:
%load_ext autoreload
%autoreload 1
%aimport data.adress
%aimport detectors.filler_speech.keyword_search
%aimport utils

In [0]:
import sys
sys.path.append("..")
import json
import numpy as np
import pandas as pd
import re
from pprint import pprint
# Evaluation
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
# Generative AI
from openai import OpenAI
from utils import llm_call
import mlflow
from mlflow.genai.scorers import Safety, scorer
from mlflow.entities import Feedback

## Load the Data

In [0]:
from data.adress import load_transcripts

In [0]:
adress_trans = load_transcripts()
adress_trans = adress_trans[["Timestamp", "Speaker", "Transcript", "Transcript_clean", "Filler"]]

# Extract annotation based count of fillers
adress_trans["num_fillers"] = adress_trans["Transcript"].apply(lambda x: len(re.findall(r"&(?!=)", x)))

adress_trans.head()

In [0]:
trn_pt_utt_idx = (adress_trans.index.get_level_values("split") == "train") & (adress_trans["Speaker"] == "Patient")
dev_pt_utt_idx = (adress_trans.index.get_level_values("split") == "dev")   & (adress_trans["Speaker"] == "Patient")
tst_pt_utt_idx = (adress_trans.index.get_level_values("split") == "test")  & (adress_trans["Speaker"] == "Patient")

## Baseline: Keyword Search

In [0]:
from detectors.filler_speech.keyword_search import FillerKeywordDetector
from utils import create_custom_nlp

First, we look at the annotated fillers in the development dataset to help inform our list of keywords.

In [0]:
np.unique( np.concatenate(adress_trans.loc[dev_pt_utt_idx, "Transcript"].apply(lambda x: re.findall(r"&(\w+)\s*", x)).values) )

Then we define our keyword lists.

In [0]:
import string

In [0]:
filler_sounds = [
    "ah",
    "eh",
    "er",
    "hm",
    "huh",
    "mm",
    "uh",
    "um",
]

filler_words = [
    "like",
    "well",
    "so",
    "basically",
    "actually",
    "literally",
]

filler_phrases = [
    "you know",
    "i mean",
    "i guess",
    "you see",
]

filler_letters = list(string.ascii_lowercase)

filler_uncommonletters = list(filter(lambda c: c not in ["a", "i", "o"], filler_letters))

#### Explore different keyword sets

In [0]:
from joblib import Parallel, delayed
from tqdm import tqdm

In [0]:
configs = {
    "sounds": (filler_sounds, False),
    "letters": (filler_letters, False),
    "uncommonletters": (filler_uncommonletters, False),
    "words": (filler_words, False),
    "nonwords": ([], True),
    "phrases": (filler_phrases, False),
    "sounds+letters": (filler_sounds + filler_letters, False),
    "sounds+uncommonletters": (filler_sounds + filler_uncommonletters, False),
    "sounds+words": (filler_sounds + filler_words, False),
    "sounds+nonwords": (filler_sounds, True),
    "sounds+phrases": (filler_sounds + filler_phrases, False),
    "sounds+uncommonletters+words": (filler_sounds + filler_uncommonletters + filler_words, False),
    "sounds+uncommonletters+nonwords": (filler_sounds + filler_uncommonletters, True),
    "sounds+uncommonletters+phrases": (filler_sounds + filler_uncommonletters + filler_phrases, True),
    "sounds+words+nonwords": (filler_sounds + filler_phrases, True),
    "sounds+nonwords+phrases": (filler_sounds + filler_phrases, True),
    "sounds+uncommonletters+nonwords+words": (filler_sounds + filler_uncommonletters + filler_words, True), # new
    "sounds+uncommonletters+nonwords+phrases": (filler_sounds + filler_uncommonletters + filler_phrases, True), # new
    "sounds+letters+words+phrases+nonwords": (filler_sounds+ filler_letters + filler_words + filler_phrases, True),
    "sounds+uncommonletters+words+nonwords+phrases": (filler_sounds + filler_uncommonletters + filler_words + filler_phrases, True)
}

In [0]:
def run(cfg_name, cfg, dataset):
    # Create spaCy vocab
    nlp = create_custom_nlp()
    # Initialize keyword detector with config
    d = FillerKeywordDetector(nlp, *cfg)
    # Run detector on dataset
    outputs = dataset["Transcript_clean"].apply(d.detect)
    # Evaluate performance
    true = dataset["Filler"]
    pred = outputs.apply(lambda x: len(x["detections"]) > 0).astype(int)
    prec = precision_score(true, pred)
    rec  = recall_score(true, pred)
    f1   = f1_score(true, pred)
    acc  = accuracy_score(true, pred)
    return cfg_name, prec, rec, f1, acc

In [0]:
results = Parallel(n_jobs=10, return_as="generator_unordered")(delayed(run)(cfg_name, configs[cfg_name], adress_trans.loc[trn_pt_utt_idx]) for cfg_name in configs)
results = [res for res in tqdm(results, total=len(configs))]

table = pd.DataFrame(results, columns=["config", "precision", "recall", "f1", "accuracy"])
table.to_csv("filler_trn_results.csv")

In [0]:
table.sort_values("recall", ascending=False).round(3)
# print(table.round(3).to_latex(index=False))

#### Evaluate best filler keyword detector configuration

In [0]:
# create custom spaCy
nlp = create_custom_nlp()
# init detector
keyword_detector_final = FillerKeywordDetector(nlp, filler_sounds + filler_uncommonletters, False)

# run on all data
adress_trans["keyword_dets"] = adress_trans.apply(lambda x: keyword_detector_final.detect(x["Transcript_clean"]) if x["Speaker"] == "Patient" else pd.NA, axis=1)

###### Performance on the test data.

In [0]:
def evaluate(true, pred):
    prec = precision_score(true, pred)
    rec  = recall_score(true, pred)
    f1   = f1_score(true, pred)
    acc  = accuracy_score(true, pred)
    return f"{prec:.3f} & {rec:.3f} & {f1:.3f} & {acc:.3f} \\\\"

In [0]:
true = adress_trans.loc[tst_pt_utt_idx, "Filler"]
pred = adress_trans.loc[tst_pt_utt_idx, "keyword_dets"].apply(lambda x: len(x["detections"]) > 0).astype(int)
print(evaluate(true, pred))

## LLM-Based Detector

##### Explore different Prompts

In [0]:
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()

client = OpenAI(
    api_key=mlflow_creds.token,
    base_url=f"{mlflow_creds.host}/serving-endpoints"
)

In [0]:
# v1
'''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds, words, or phrases used to fill pauses in speech during speech planning (e.g., "uh", "like", "you know").

# OUTPUT FORMAT
Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The exact filler text that was identified.
- "span": The character span of the filler in the utterance.

# INPUT
{}
'''

# v2
'''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds, words, or phrases used to fill pauses in speech during speech planning (e.g., "uh", "like", "you know").

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The exact filler text that was identified.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

#v3
prompt = '''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds, words, or phrases used to fill pauses in speech during speech planning (e.g., "uh", "like", "you know"). Do not flag word repetiton as filler. Do not flag event tags of the format "[<event>]" as filler.

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The exact filler text that was identified.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

#v4
'''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds, words, or phrases used to fill pauses in speech during speech planning (e.g., "uh", "like", "you know"). Do not flag word repetiton as filler. Do not flag event tags of the format "[<event>]" as filler. Do not flag event tags of the format "[<event>]" as filler. Do not flag word repetiton as filler. Every detected filler MUST be an exact, verbatim substring of the input utterance.

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The exact text of the detected filler from the utterance.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

#v5
'''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds, words, or phrases used to fill pauses in speech during speech planning (e.g., "uh", "like", "you know"). Do not flag word repetiton as filler. Do not flag event tags of the format "[<event>]" as filler.

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The verbatim substring from the utterance that was identified as the filler.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

#v6
'''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words -- such as "uh", "like", "you know" -- are sounds, words, or phrases used to fill pauses in speech while formulating what to say next. Do not flag word repetiton as filler. Do not flag event tags of the format "[<event>]" as filler.

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The verbatim substring from the utterance that was identified as the filler.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

#v7
prompt = '''# INSTRUCTIONS
You are a neurologist analyzing a patient's speech sample for signs of cognitive impairment. Your task is to identify the use of filler words in the provided utterance below. Filler words are sounds (like "uh", "um"), fragments (like "r", "sh"), words (like "well", "like"), or phrases (like "I mean", "you know") used to fill pauses in speech while formulating what to say next. Do not flag word repetiton as filler. Do not flag event tags of the format "[<event>]" as filler.

Your output must be a single JSON object with a single key "detections" whose value is an array of JSON objects. Each object in the array represents one detected filler and must have the following three keys-value pairs:
- "type": "filler".
- "text": The verbatim substring from the utterance that was identified as the filler.
- "span": The character span of the filler in the utterance.

# UTTERANCE
{}
'''

In [0]:
trn_dataset = []
for idx, row in adress_trans.loc[trn_pt_utt_idx].iterrows():
    trn_dataset.append({
        "split": idx[0],
        "ID": idx[1],
        "utt_num": idx[2],
        "inputs": {"text": row["Transcript_clean"]},
        "expectations": {"has_filler": bool(row["Filler"])}
    })

In [0]:
@scorer
def correct(expectations, outputs):
    return Feedback(value=(expectations["has_filler"] == (len(outputs["detections"]) > 0)))

In [0]:
fn = lambda text: llm_call(client, "openai_gpt_4o", None, prompt.format(text), {"type": "json_object"})

with mlflow.start_run(run_name="llm_trn_gpt4o_pV7_2") as run:
    gpt_dets_trn = mlflow.genai.evaluate(
        predict_fn=fn,
        data=trn_dataset,
        scorers=[correct]
    )

In [0]:
def process_mlflow_outputs(result):
    outputs = result.tables["eval_results"][["response", "assessments"]]
    outputs["pred"] = outputs["response"].apply(lambda x: (len(x["detections"]) > 0) if type(x) == dict else False).astype(int)
    outputs["label"] = outputs["assessments"].apply(lambda x: [a["value"] for a in x if a["name"] == "has_filler"][0]).astype(int)
    return outputs

In [0]:
outputs = process_mlflow_outputs(gpt_dets_trn)
print(evaluate(outputs["label"], outputs["pred"]))

##### Compare performance of different LLMs

In [0]:
fn = lambda text: llm_call(client, "meta_llama_v3_1_8b_instruct", None, prompt.format(text), {"type": "json_object"})

with mlflow.start_run(run_name="llm_eval_llama3_1_8b_pV7_2") as run:
    llama_dets_trn = mlflow.genai.evaluate(
        predict_fn=fn,
        data=trn_dataset,
        scorers=[correct]
    )

In [0]:
outputs = process_mlflow_outputs(llama_dets_trn)
print(evaluate(outputs["label"], outputs["pred"]))

Llama actually performs extremely terrible with this prompt.

##### Evaluate the best configuration

In [0]:
tst_dataset = []
for idx, row in adress_trans.loc[tst_pt_utt_idx].iterrows():
    tst_dataset.append({
        "split": idx[0],
        "ID": idx[1],
        "utt_num": idx[2],
        "inputs": {"text": row["Transcript_clean"]},
        "expectations": {"has_filler": bool(row["Filler"])}
    })

In [0]:
fn = lambda text: llm_call(client, "openai_gpt_4o", None, prompt.format(text), {"type": "json_object"})

with mlflow.start_run(run_name="gpt_eval_tst") as run:
    gpt_dets_tst = mlflow.genai.evaluate(
        predict_fn=fn,
        data=tst_dataset,
        scorers=[correct]
    )

In [0]:
outputs = process_mlflow_outputs(gpt_dets_tst)
print(evaluate(outputs["label"], outputs["pred"]))

## (@Sriharsha) Try using a LLM critic to post-process keyword detections.
We are already acheiving high performance with the keyword detector using filler sounds and uncommon letters. Can we improve performance by using an LLM to remove false positive detections?

*Experiment*: For each patient session, aggregate the utterances into a transcript and the filler detection lists (remember to offset them). Query an LLM to remove detections that are not actually filler. Try 3 different LLMs once you've settled on a good prompt. Also, try using MLFlow to log LLM outputs. It makes comparing the effect of different prompt versions a little easier to visualize.

##### Investigate the failure cases or prior methods.

Keyword search detector

In [0]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    print("False positives:")
    pprint(adress_trans.loc[dev_pt_utt_idx & (adress_trans["Filler"] == 0) & (adress_trans["keyword_dets"].apply(lambda x: len(x["detections"]) > 0 if type(x) == dict else False)), ["Transcript", "keyword_dets"]].values)

In [0]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    print("False negatives:")
    pprint(adress_trans.loc[dev_pt_utt_idx & (adress_trans["Filler"] == 1) & (adress_trans["keyword_dets"].apply(lambda x: len(x["detections"]) == 0 if type(x) == dict else False)), ["Transcript", "keyword_dets"]].values)

LLM-based detector

In [0]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    print("False positives:")
    pprint(adress_trans.loc[dev_pt_utt_idx & (adress_trans["Filler"] == 0) & (adress_trans["llm_dets"].apply(lambda x: len(x["detections"]) > 0 if type(x) == dict else False)), ["Transcript", "llm_dets"]].values)

In [0]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    print("False negatives:")
    pprint(adress_trans.loc[dev_pt_utt_idx & (adress_trans["Filler"] == 1) & (adress_trans["llm_dets"].apply(lambda x: len(x["detections"]) == 0 if type(x) == dict else False)), ["Transcript", "llm_dets"]].values)

##### Explore different prompts.

In [0]:
# @Sriharsha Have the llm output a json object with the key "fillers" whose value is a list of json objects for each detection. Each detection object must have a "text" key whose value is the filler sound/word/phrase and a "span" key whose value is a list of the start and end character index of the filler in the utterance.

critic_prompt = '''{}'''

In [0]:
dataset_kw = []
dataset_llm = []
dataset = []

In [0]:
fn = lambda transcript, detections: llm_call(client, "openai_gpt_4o", None, critic_prompt.format(transcript, detections), {"type": "json_object"})

with mlflow.start_run(run_name="Keywords + Critic pv0") as run:
    result = mlflow.genai.evaluate(
        predict_fn=fn,
        data=dataset,
        scorers=[correct]
    )

##### Evaluate the best prompt

In [0]:
fn = lambda text: llm_call(client, "openai_gpt_4o", None, critic_prompt.format(text), {"type": "json_object"})

with mlflow.start_run(run_name="LLM + Critic pv0") as run:
    result = mlflow.genai.evaluate(
        predict_fn=fn,
        data=dataset_kw,
        scorers=[correct]
    )

In [0]:
fn = lambda text: llm_call(client, "openai_gpt_4o", None, critic_prompt.format(text), {"type": "json_object"})

with mlflow.start_run(run_name="LLM + Critic pv0") as run:
    result = mlflow.genai.evaluate(
        predict_fn=fn,
        data=dataset_llm,
        scorers=[correct]
    )

## Summary metrics for filler detections

In [0]:
from data.adress import load_outcomes

In [0]:
outcomes = load_outcomes()
outcomes.head()

In [0]:
tst_pts = outcomes.loc[(outcomes.index.get_level_values("split") == "test")].index.values
tst_ad_pts = outcomes.loc[(outcomes.index.get_level_values("split") == "test") & (outcomes["AD_dx"] == 1)].index.values
tst_cn_pts = outcomes.loc[(outcomes.index.get_level_values("split") == "test") & (outcomes["AD_dx"] == 0)].index.values

**Filler Rate** = Total number of detected fillers / Total number of words spoken

In [0]:
def compute_filler_rate(outputs):
    num = outputs.apply(lambda x: len(x["detections"]) if not pd.isna(x) else 0).groupby(level=("split", "ID")).sum()
    den = adress_trans.apply(lambda x: sum([1 for token in nlp(x["Transcript_clean"]) if not (token.is_punct or token.is_space or token._.is_silence_tag or token._.is_inaudible_tag or token._.is_event_tag)]) if x["Speaker"] == "Patient" else 0, axis=1).groupby(level=("split", "ID")).sum()
    return 100 * num / den

In [0]:
outcomes["gt_filler_rate"] = compute_filler_rate(adress_trans["num_fillers"].apply(lambda x: {"detections": [0] * x}))
outcomes["kw_filler_rate"] = compute_filler_rate(adress_trans["keyword_dets"])

In [0]:
plt.violinplot(
    (outcomes.loc[tst_ad_pts, "gt_filler_rates"],
     outcomes.loc[tst_cn_pts, "gt_filler_rates"],
     outcomes.loc[tst_ad_pts, "kw_filler_rates"],
     outcomes.loc[tst_cn_pts, "kw_filler_rates"]),
    showmedians=True,
)
plt.xticks(range(1, 5), ["GT AD Group", "GT Control Group", "KW AD Group", "KW Control Group"])
plt.ylim([0, 100])
plt.ylabel("Filler Rate")
plt.grid()
plt.show()

**Inter filler Distance**: The mean and standard deviation of the number of words between fillers within utterances

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
def inter_filler_dist(output_name):
    ifd_metrics = pd.DataFrame(index=outcomes.index, columns=["mean_IFD", "std_IFD", "mean_IFD_norm", "mean_IFD_imputed", "std_IFD_imputed"], dtype=float)

    for split, pt_id in ifd_metrics.index:
        ifds = []
        for utt_num, row in adress_trans.loc[(split, pt_id)].iterrows():
            if row["Speaker"] == "Patient":     # skip provider speech
                if len(row[output_name]["detections"]) > 1:     # can only compute IFD if there is more than one filler
                    # get word spans for utterance
                    doc = nlp(row["Transcript_clean"])
                    word_spans = [(token.text, token.idx, token.idx + len(token.text)) for token in doc if not (token.is_punct or token.is_space or token._.is_silence_tag or token._.is_inaudible_tag or token._.is_event_tag)]
                    # print("word_spans", word_spans)

                    # get filler words indices
                    filler_word_idxs = [word_spans.index((det["text"], det["span"][0], det["span"][1])) for det in row[output_name]["detections"]]
                    # print("filler_word_idxs:", filler_word_idxs)

                    # inter filler distance
                    ifds.extend([filler_word_idxs[i+1] - filler_word_idxs[i] - 1 for i in range(len(filler_word_idxs) - 1)])
                    # print("ifd", ifds[-1])
                    # break

        ifd_metrics.loc[(split, pt_id), "mean_IFD"] = np.mean(ifds)
        ifd_metrics.loc[(split, pt_id), "std_IFD"] = np.std(ifds)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(ifd_metrics.loc["train", ["mean_IFD"]])
    ifd_metrics["mean_IFD_norm"] = scaler.transform(ifd_metrics[["mean_IFD"]])
    ifd_metrics["mean_IFD_imputed"] = ifd_metrics["mean_IFD_norm"].fillna(1.0)
    ifd_metrics["std_IFD_imputed"] = ifd_metrics["std_IFD"].fillna(0)

    return ifd_metrics                            

In [0]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    outcomes[["kw_mean_IFD", "kw_std_IFD", "kw_mean_IFD_imp", "kw_std_IFD_imp"]] = inter_filler_dist("keyword_dets")[["mean_IFD", "std_IFD", "mean_IFD_imputed", "std_IFD_imputed"]]

In [0]:
plt.violinplot(
    (outcomes.loc[tst_ad_pts, "kw_mean_IFD"].dropna(),
     outcomes.loc[tst_cn_pts, "kw_mean_IFD"].dropna(),
     outcomes.loc[tst_ad_pts, "kw_std_IFD"].dropna(),
     outcomes.loc[tst_cn_pts, "kw_std_IFD"].dropna()),
    showmedians=True,
)
plt.xticks(range(1, 5), ["Mean AD Group", "Mean Control Group", "Std AD Group", "Std Control Group"], rotation=45, ha="right")
plt.ylabel("Inter Filler Distance")
plt.grid()
plt.show()
plt.close()

Analyze correlation between our metrics and the outcome variable

In [0]:
metrics = [
    "gt_filler_rate", 
    "kw_filler_rate", 
    "kw_mean_IFD", 
    "kw_std_IFD",
]

In [0]:
for m in metrics:
    ## score averages
    mean_ad = outcomes.loc[tst_ad_pts, m].mean()
    std_ad = outcomes.loc[tst_ad_pts, m].std()
    mean_cn = outcomes.loc[tst_cn_pts, m].mean()
    std_cn = outcomes.loc[tst_cn_pts, m].std()

    ## correlation metrics
    match = re.search(r"(\w+)_(\w+)_IFD", m)
    if match:
        # use imputed feature values for statistical tests
        m = "%s_%s_IFD_imp" % match.groups()

    res_ttest = ttest_ind(outcomes.loc[tst_pts, "AD_dx"], outcomes.loc[tst_pts, m])
    res_mannw = mannwhitneyu(outcomes.loc[tst_pts, "AD_dx"], outcomes.loc[tst_pts, m])
    res_auc   = roc_auc_score(outcomes.loc[tst_pts, "AD_dx"], outcomes.loc[tst_pts, m])

    print("%s & %.2f (%.2f) & %.2f (%.2f) & %.2f (%s) & %.2f (%s) & %.2f \\\\" % 
            (m,
            mean_ad,
            std_ad,
            mean_cn,
            std_cn,
            res_ttest.statistic,
            str(round(res_ttest.pvalue, 3)) if res_ttest.pvalue >= 0.001 else "$<$0.001", 
            res_mannw.statistic, 
            str(round(res_ttest.pvalue, 3)) if res_ttest.pvalue >= 0.001 else "$<$0.001",
            res_auc)
    )

In [0]:
outcomes[["kw_filler_rate", "kw_mean_IFD_imp", "kw_std_IFD_imp"]].to_csv("filler_feats.csv")